In [1]:
import tensorflow as tf
msg = tf.constant('Hello, TensorFlow!... finally')
tf.print(msg)

%matplotlib inline
import re
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt

from keras import models
from keras import layers
from keras.utils import to_categorical
from keras.datasets import imdb
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

Hello, TensorFlow!... finally


Using TensorFlow backend.


In [3]:
n = ['id', 'date', 'name', 'text', 'typr', 'rep', 'rtw', 'faw', 'stcount', 'foll', 'frien', 'listcount']

data_positive = pd.read_csv('positive.csv', sep=';', error_bad_lines=False, names=n, usecols=['text'])
data_negative = pd.read_csv('negative.csv', sep=';', error_bad_lines=False, names=n, usecols=['text'])

sample_size = min(data_positive.shape[0], data_negative.shape[0])
raw_data = np.concatenate((data_positive['text'].values[:sample_size],
                           data_negative['text'].values[:sample_size]), axis=0)
labels = [1] * sample_size + [0] * sample_size

#data_positive['text'][1243]

def preprocess_text(text):
    text = text.lower().replace("ё", "е")
    #replace URL an NICKNAME
    text = re.sub(r'((www\.[^\s]+)|(https?://[^\s]+))', 'URL', text)
    text = re.sub(r'@[^\s]+', 'USER', text)
    #delete words with letters-symbols and lang mix
    text = re.sub(r'[\W]+', ' ', text)
    #delete predlogi
    #text = re.sub(r'\b[\w]{,3}\b', '', text)
    text = re.sub(r'\b[^не]{,3}\b', ' ', text)
    #delete tab symbols
    text = re.sub(r'[\s]+', ' ', text)
    return text.strip()

data_int = [preprocess_text(t) for t in raw_data]


length = [len(i) for i in data_int]
print("Average Review length:", np.mean(length))
print("Standard Deviation:", round(np.std(length)))

Average Review length: 57.75573832009506
Standard Deviation: 25.0


In [6]:
foo='z USER z ахахах ну ты ж ее стааарше меня мне нужна поддержка ты ж не t против кому если не тебе a'
#foo=raw_data[1]

i=98
#print(raw_data[i])
#print(data_int[i])
print(foo)
print(re.sub(r'\b[^не]{,3}\b', ' ', foo))

#print([index.get(str,0) for str in data_int[i].split(' ')])

z USER z ахахах ну ты ж ее стааарше меня мне нужна поддержка ты ж не t против кому если не тебе a
  USER  ахахах  ну   ее  стааарше  меня  мне  нужна  поддержка   не  против  кому  если  не  тебе  


In [7]:
SENTENCE_LENGTH = 50
NUM = 10000
NUM_TEST = 50000

def get_sequences(data_p):
    sequences = tokenizer.texts_to_sequences(data_p)
    return pad_sequences(sequences, maxlen=SENTENCE_LENGTH)

#we will use only NUM top used words as comp nerezinovi
def get_my_sequences(comment_p):
    result = [index.get(str,1) for str in comment_p.split(' ') if index.get(str,1) < NUM]
    return result

def vectorize(sequences, dimension = NUM):
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        results[i, sequence] = 1
    return results


tokenizer = Tokenizer(num_words=NUM,oov_token="#")
tokenizer.fit_on_texts(data_int[NUM:])

index = tokenizer.word_index

#Change words to indexes from vocabulary that we sdelali
data = [get_my_sequences(comment) for comment in data_int]

#Change to 0-1 index
data = vectorize(data)
targets = np.array(labels).astype("float32")
        
#this is keras train/test it is with shuffle
train_x, test_x, train_y, test_y = train_test_split(data, labels, test_size=0.2, random_state=2)

#test_x = data[:NUM_TEST]
#test_y = targets[:NUM_TEST]

#train_x = data[NUM_TEST:]
#train_y = targets[NUM_TEST:]

In [8]:
i=3
ttt1 = [index.get(str,1) for str in data_int[i].split(' ')]
print(index.get('школота'))
print(index.get('поверь'))
print(len(index))

print('data:_____')
print(data_int[i])
print(ttt1)
print(get_my_sequences(data_int[i]))
print(data[i][0:60])


13866
2892
170145
data:_____
USER углу сидит погибает голода еще порции взяли хотя уже жрать не хотим
[1, 5005, 759, 163604, 6285, 10, 16649, 1653, 97, 11, 889, 3, 2216]
[1, 5005, 759, 6285, 10, 1653, 97, 11, 889, 3, 2216]
[0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


-3 is some magic.# replace when word not found

In [9]:
model = models.Sequential()
# Input - Layer
model.add(layers.Dense(50, activation = "relu", input_shape=(10000, )))
# Hidden - Layers
model.add(layers.Dropout(0.3, noise_shape=None, seed=None))
model.add(layers.Dense(50, activation = "relu"))
model.add(layers.Dropout(0.2, noise_shape=None, seed=None))
model.add(layers.Dense(50, activation = "relu"))
# Output- Layer
model.add(layers.Dense(1, activation = "sigmoid"))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 50)                500050    
_________________________________________________________________
dropout_1 (Dropout)          (None, 50)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 50)                2550      
_________________________________________________________________
dropout_2 (Dropout)          (None, 50)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 50)                2550      
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 51        
Total params: 505,201
Trainable params: 505,201
Non-trainable params: 0
________________________________________________

In [10]:
model.compile(
 optimizer = "adam",
 loss = "binary_crossentropy",
 metrics = ["accuracy"]
)

In [11]:
results = model.fit(
 train_x, train_y,
 epochs= 2,
 batch_size = 500,
 validation_data = (test_x, test_y)
)


Train on 179076 samples, validate on 44770 samples
Epoch 1/2
179076/179076 [==============================] - 60s 336us/step - loss: 0.5735 - accuracy: 0.6967 - val_loss: 0.5330 - val_accuracy: 0.7306
Epoch 2/2
179076/179076 [==============================] - 64s 360us/step - loss: 0.4992 - accuracy: 0.7544 - val_loss: 0.5285 - val_accuracy: 0.7341


In [12]:
print(np.mean(results.history["val_accuracy"]))


0.7323542535305023


In [13]:
testPositive = "прекрасный наполненный смыслом и радостью фильм понравился очень"
testNegative = "этот фильм полный шлак посмотрел самый худший фильм видел оценка минус один нуля"

print(testPositive)
print(testNegative)

testPositiveArr = [1]
testNegativeArr = [1]

print("\n"+"Get index "+"\n")

testPositiveArr[0] = get_my_sequences(testPositive)
testNegativeArr[0] = get_my_sequences(testNegative)

print(testPositiveArr)
print(testNegativeArr)

прекрасный наполненный смыслом и радостью фильм понравился очень
этот фильм полный шлак посмотрел самый худший фильм видел оценка минус один нуля

Get index 

[[1405, 1, 8683, 2385, 179, 936, 18]]
[[68, 179, 1084, 784, 182, 6773, 179, 493, 5039, 1132, 82]]


In [14]:
testPositiveArrINDX = vectorize(testPositiveArr)
testNegativeArrINDX = vectorize(testNegativeArr)

predictPositive = model.predict(testPositiveArrINDX)
predictNegative = model.predict(testNegativeArrINDX)

print('Positive: ' + str(predictPositive[0][0]))
print('Negative: ' + str(predictNegative[0][0]))

Positive: 0.8635264
Negative: 0.051291257


In [15]:
model.save('trainedModel.hdf5')
df = pd.DataFrame.from_dict(index, orient="index")
df.to_csv("commentDict.csv")